In [14]:
%load_ext autoreload
%autoreload 2
import concurrent.futures
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize

notebook_name = "gridrad_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T23:00:00"
end = "2010-01-21T00:00:00"
event_start = "2010-01-20"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / "runs/gridrad_demo"
if output_parent.exists():
    shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options, event_start=event_start
)

lon_range = [-102, -89]
lat_range = [27, 39]
era5_pl_options = data.era5.data_options(
    start=start, end=end, longitude_range=lon_range, latitude_range=lat_range
)
args_dict = {"start": start, "end": end, "data_format": "single-levels"}
args_dict.update({"longitude_range": lon_range, "latitude_range": lat_range})
era5_sl_options = data.era5.data_options(**args_dict)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory=options_directory)

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.mcs(
    dataset="gridrad",

    global_flow_margin=70,
    unique_global_flow=False,
)

option.check_options(track_options)
option.save_track_options(track_options, options_directory=options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
# visualize_options = None

2024-10-09 09:18:41,905 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/data.yml
2024-10-09 09:18:41,921 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-09 09:18:41,922 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/grid.yml
2024-10-09 09:18:41,928 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/track.yml


In [16]:
times = data.utils.generate_times(data_options["gridrad"])
track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_parent
)

2024-10-09 09:18:55,420 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo.
2024-10-09 09:18:55,421 - thor.track - INFO - Beginning simultaneous tracking.


2024-10-09 09:18:55,454 - thor.track - INFO - Processing 2010-01-20T23:00:00.
2024-10-09 09:18:55,455 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T23:00:00.
2024-10-09 09:18:55,456 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T230000Z.nc
2024-10-09 09:18:55,457 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T23:00:00.000000000.
2024-10-09 09:18:55,615 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-09 09:18:56,406 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-09 09:18:56,462 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-09 09:18:56,784 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-10-09 09:18:57,033 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-09 09:18:57,342 - thor.data.gridrad - DEBUG - Got domain mask for gridrad.
2024-10-09 09:18:57,828 - thor.data.gr

In [3]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = []
    for i, time_interval in enumerate(intervals):
        args = [i, time_interval, data_options, grid_options, track_options]
        args += [visualize_options, output_parent, "gridrad"]
        futures.append(executor.submit(parallel.track_interval, *args))
    parallel.check_futures(futures)

2024-10-08 21:32:33,151 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_3/options/data.yml
2024-10-08 21:32:33,151 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_2/options/data.yml
2024-10-08 21:32:33,151 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_1/options/data.yml
2024-10-08 21:32:33,156 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_0/options/data.yml
2024-10-08 21:32:33,709 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_1/options/grid.yml
2024-10-08 21:32:33,723 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_1/options/track.yml
2024-10-08 21:32:33,743 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_3/options/grid.yml
2024-10-08 21:32:33,755 - thor.track - INFO - Beginnin

In [4]:
parallel.stitch_run(output_parent, intervals, cleanup=True)

2024-10-08 21:37:25,475 - thor.parallel - INFO - Stitching all attribute, mask and record files.
2024-10-08 21:37:28,872 - thor.parallel - INFO - Stitching record files.
2024-10-08 21:37:28,904 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/records/filepaths/gridrad.csv
2024-10-08 21:37:28,908 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/records/filepaths/gridrad.yml
2024-10-08 21:37:28,910 - thor.parallel - INFO - Stitching attribute files.
2024-10-08 21:37:28,999 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/attributes/anvil/core.csv
2024-10-08 21:37:29,009 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/attributes/anvil/core.yml
2024-10-08 21:37:29,085 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/a

In [5]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent)

2024-10-08 21:37:33,793 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.csv
2024-10-08 21:37:33,800 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.yml
2024-10-08 21:37:33,987 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.csv
2024-10-08 21:37:33,991 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.yml
2024-10-08 21:37:34,043 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.csv
2024-10-08 21:37:34,045 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.yml


stratiform_offset inflow  \
time                universal_id                            
2010-01-20 18:00:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:10:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:20:00 1                      leading  right   
...                                            ...    ...   
2010-01-21 03:10:00 23                       right  front   
                    25                       right  front   
                    30                    trailing  front   
2010-01-21 03:20:00 23                       right  front   
                    25                     leading  front   

                                 relative_stratiform_offset  \
time                universal_id                              
2010-01-20 18:00:00 1                                  left   
                    2                                  left   
2010-01-20 18:10:00 1                                  left   
                    2                                  left   
2010-01-20 18:20:00 1                                  left   
...                                                     ...   
2010-01-21 03:10:00 23                                right   
                    25                              leading   
                    30                             trailing   
2010-01-21 03:20:00 23                                right   
                    25                              leading   

                                                 tilt          propagation  
time                universal_id                                            
2010-01-20 18:00:00 1                      down-shear           down-shear  
                    2                      down-shear  shear-perpendicular  
2010-01-20 18:10:00 1                      down-shear           down-shear  
                    2                      down-shear  shear-perpendicular  
2010-01-20 18:20:00 1                      down-shear           down-shear  
...                                               ...                  ...  
2010-01-21 03:10:00 23            shear-perpendicular           down-shear  
                    25                     down-shear           down-shear  
                    30                       up-shear           down-shear  
2010-01-21 03:20:00 23            shear-perpendicular           down-shear  
                    25                     down-shear           down-shear  

[248 rows x 5 columns]

In [7]:
%load_ext memory_profiler

In [6]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64("2010-01-20T21:50")
end_time = np.datetime64(np.datetime64("2010-01-21T03:30"))
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 5400, "by_date": False}
visualize.attribute.mcs_series(*args, **args_dict)
# %mprun -f visualize.attribute.mcs_series visualize.attribute.mcs_series(*args, **args_dict)

2024-10-08 21:37:34,144 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T21:50:00.000000000.
2024-10-08 21:37:34,298 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-08 21:37:35,082 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-08 21:37:35,156 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-08 21:37:35,514 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-10-08 21:37:35,801 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-08 21:37:36,096 - thor.data.gridrad - DEBUG - Got domain mask for gridrad.
2024-10-08 21:37:36,199 - thor.visualize.attribute - DEBUG - Getting grid from dataset at time 2010-01-20T21:50:00.000000000.
2024-10-08 21:37:36,211 - thor.data.gridrad - DEBUG - Got grid from dataset at time 2010-01-20T21:50:00.000000000.
2024-10-08 21:37:36,213 - thor.visualize.attribute - DEBUG - Rebuilding processed grid for time

In [ ]:
import xarray as xr
import requests
from io import BytesIO

# URL to the remote NetCDF file
url = "https://data.rda.ucar.edu/ds841.6/2010/20100120/nexrad_3d_v4_2_20100120T230000Z.nc"

# API key or other authentication details
headers = {
    "Authorization": "Bearer YOUR_API_KEY"
}

# Make the request to get the file content
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful

# Open the dataset using xarray
file_obj = BytesIO(response.content)
ds = xr.open_dataset(file_obj, engine="netcdf4")

# Perform your analysis
print(ds)

# Example: Accessing a variable
temperature = ds["temperature"]
print(temperature)